# Proyecto

Libraries

In [1]:
import serial
import sys
import glob
import time

Functions

In [2]:
def listSerialPorts():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

def connectToPort(n):
    ports = listSerialPorts()
    return serial.Serial(ports[n])

def readValueFromData(data):
    # +000.000/0 G S
    dataString = data.decode("utf-8").replace('\r\n','')
    if dataString.endswith(' G S'):
        result = dataString.replace(' G S','').replace('/','').replace('+','')
        return float(result)
    else:
        raise Exception("Invalied format")

        
    

Connection

In [16]:

ports = listSerialPorts()

for i in range(0,len(ports)):
    print('{0}: {1}'.format(i, ports[i]))
connection = connectToPort(3)

0: /dev/tty.wlan-debug
1: /dev/tty.BT-163
2: /dev/tty.Bluetooth-Incoming-Port
3: /dev/tty.usbserial-FTDCYOPE
Serial<id=0x107e99a20, open=True>(port='/dev/tty.usbserial-FTDCYOPE', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False)


In [17]:
for i in range(0,10):
    error = True
    while error:
          try:
              connection.flushInput()
              value = readValueFromData(connection.readline())
              print(value)
              error = False
          except:
              pass
    time.sleep(1)

2.0
0.0001
200.0001
200.0001
200.0
200.0
200.0
200.0
200.0
200.0


In [18]:
connection.close()